In [1]:
import numpy as np
import itertools as it
import matplotlib.pyplot as plt

# Global variable for the number of pairs
num_pairs = 2

def base(M, n):
    # calculate the image of the base under a matrix M
    # M: Binary matrix
    # n: number of pairs

    # make a set of all combinations of the first column and the last n columns (these correspond to X_1, Z_1,...,Z_n)
    s = []
    for i in range(n+1, 2*n):
        s.append(M[0:2*n, i])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))
    
    res = [vector(GF(2),2*n)]
        
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        res.append(v)
        
    return res


def pillars(M, n):
    # calculate the image of the pillars under a matrix M
    # M: Binary matrix
    # n: number of pairs
    X1 = vector(M[0:2*n, 0])
    Z1 = vector(M[0:2*n, n])
    Y1 = X1 + Z1
    
    pI = base(M, n)
    pX = [(X1 + b) for b in pI]
    pY = [(Y1 + b) for b in pI]
    pZ = [(Z1 + b) for b in pI]
    
    return [pI, pX, pY, pZ]   


def generate_pauli_dict(p_I, p_X, p_Y, p_Z):

    # Given a Pauli vector return a dictionary in {"XX": p_X*p_X, ...}. This is a representation of a state \psi \otimes \psi.

    paulis = ["I", "X", "Y", "Z"]
    result = {}
    for p1 in paulis:
        for p2 in paulis:
            value = 1
            if p1 == "I":
                value *= p_I
            if p1 == "X":
                value *= p_X
            if p1 == "Y":
                value *= p_Y
            if p1 == "Z":
                value *= p_Z
            if p2 == "I":
                value *= p_I
            if p2 == "X":
                value *= p_X
            if p2 == "Y":
                value *= p_Y
            if p2 == "Z":
                value *= p_Z
            result[p1+p2] = value      
    return result  


def bin_to_pauli(input):
    # Convert a binary string to correspond Pauli operator, also in string.
    if input == "00":
        return "I"
    elif input == "10":
        return "X"
    elif input == "01":
        return "Z"
    elif input == "11":
        return "Y"
    
def translate_pauli(pillars):
    # Given pillars, get the Pauli elements in string format.
    # For example, if DEJMPS matrix is given, this returns [["II", "YY"], ["XX", "ZZ"], ["ZX", "XZ"], ["YI", "IY"]]
    pilI = []
    pilX = []
    pilY = []
    pilZ = []

    #Case: pilI
    pilI.append(bin_to_pauli(str(pillars[0][0][0]) + str(pillars[0][0][2])) + bin_to_pauli(str(pillars[0][0][1]) + str(pillars[0][0][3]))) 
    pilI.append(bin_to_pauli(str(pillars[0][1][0]) + str(pillars[0][1][2])) + bin_to_pauli(str(pillars[0][1][1]) + str(pillars[0][1][3]))) 

    #Case: pilX
    
    pilX.append(bin_to_pauli(str(pillars[1][0][0]) + str(pillars[1][0][2])) + bin_to_pauli(str(pillars[1][0][1]) + str(pillars[1][0][3]))) 
    pilX.append(bin_to_pauli(str(pillars[1][1][0]) + str(pillars[1][1][2])) + bin_to_pauli(str(pillars[1][1][1]) + str(pillars[1][1][3]))) 

    #Case: pilY
    pilY.append(bin_to_pauli(str(pillars[2][0][0]) + str(pillars[2][0][2])) + bin_to_pauli(str(pillars[2][0][1]) + str(pillars[2][0][3]))) 
    pilY.append(bin_to_pauli(str(pillars[2][1][0]) + str(pillars[2][1][2])) + bin_to_pauli(str(pillars[2][1][1]) + str(pillars[2][1][3]))) 

    #Case: pilZ
    pilZ.append(bin_to_pauli(str(pillars[3][0][0]) + str(pillars[3][0][2])) + bin_to_pauli(str(pillars[3][0][1]) + str(pillars[3][0][3]))) 
    pilZ.append(bin_to_pauli(str(pillars[3][1][0]) + str(pillars[3][1][2])) + bin_to_pauli(str(pillars[3][1][1]) + str(pillars[3][1][3]))) 
    
    return [pilI, pilX, pilY, pilZ]


def sum_up_dict(pauli_dict, lst_pauli_str):
    # Given a Pauli dictionary {"XX": ..., "XY": ..., ...} and a list of Pauli string ["XX": ,"YY", ...]
    # return the sum of corresponding values.
    result = 0
    for p in lst_pauli_str:
        result += pauli_dict[p]
    return result

def get_components_psuc(input_vector, M):
    # Given an input Pauli vector, and a clifford operator. Get Pauli vector after distillation circuit M.
    pauli_dict = generate_pauli_dict(input_vector[0], input_vector[1], input_vector[2], input_vector[3])
    pil_str = translate_pauli(pillars(M, num_pairs)) 
    p_I = sum_up_dict(pauli_dict, pil_str[0]) / sum_up_dict(pauli_dict, list(it.chain.from_iterable(pil_str))) 
    p_X = sum_up_dict(pauli_dict, pil_str[1]) / sum_up_dict(pauli_dict, list(it.chain.from_iterable(pil_str))) 
    p_Y = sum_up_dict(pauli_dict, pil_str[2]) / sum_up_dict(pauli_dict, list(it.chain.from_iterable(pil_str))) 
    p_Z = sum_up_dict(pauli_dict, pil_str[3]) / sum_up_dict(pauli_dict, list(it.chain.from_iterable(pil_str))) 
    
    #list(it.chain.from_iterable...) seems to be the p_suc
    p_suc = sum_up_dict(pauli_dict, list(it.chain.from_iterable(pil_str)))
    return [p_I, p_X, p_Y, p_Z], p_suc

def get_werner(fidelity):
    # Generate Werner state in Pauli vector form.
    return [fidelity, (1-fidelity) / 3, (1-fidelity) / 3, (1-fidelity) / 3]

def permute_paulies(p_vector):
    P_1 = [p_vector[0], p_vector[1], p_vector[2], p_vector[3]]
    P_2 = [p_vector[0], p_vector[1], p_vector[3], p_vector[2]]
    P_3 = [p_vector[0], p_vector[2], p_vector[1], p_vector[3]]
    P_4 = [p_vector[0], p_vector[2], p_vector[3], p_vector[1]]
    P_5 = [p_vector[0], p_vector[3], p_vector[1], p_vector[2]]
    P_6 = [p_vector[0], p_vector[2], p_vector[3], p_vector[1]]

    return [P_1, P_2, P_3, P_4, P_5, P_6]

def bin_entropy(p):
    if p == 0 or p == 1:
        return 0
    else:
        return (-p* np.log2(p)) - (1-p) * np.log2(1-p)
def get_bb84_rate(p_vector, p_suc):
    
    error_bitfilip = p_vector[1] + p_vector[2]
    error_phaseflip= p_vector[2] + p_vector[3]
    pre_factor = 1/2
    r_sk = (1 - bin_entropy(error_bitfilip) - bin_entropy(error_phaseflip)) * p_suc * pre_factor

    if r_sk < 0 :
        return 0 

    return r_sk

In [2]:
num_ED = 15

input_fid_list = np.linspace(0.25, 1, 100)

output_fids_ED_only = np.zeros((len(input_fid_list), num_ED))
output_keyrates_ED_only = np.zeros((len(input_fid_list), num_ED))
output_p_suc_ED_only = np.zeros((len(input_fid_list), num_ED))
for idx_fidelity in range(len(input_fid_list)):
    werner_state = get_werner(input_fid_list[idx_fidelity])
    for idx_ED in range(num_ED):
        M_ED = load(f'ED_transversal/2_pair/ED2_{idx_ED}.sobj').inverse()
        state_ED, p_suc = get_components_psuc(werner_state, M_ED)
        
        output_fids_ED_only[idx_fidelity, idx_ED] = state_ED[0]
        output_keyrates_ED_only [idx_fidelity, idx_ED] = get_bb84_rate(state_ED, p_suc)
        output_p_suc_ED_only [idx_fidelity, idx_ED] = p_suc

np.save("output_fids_2", output_fids_ED_only)
np.save("output_p_suc_2", output_p_suc_ED_only)
np.save("output_keyrates_2", output_keyrates_ED_only)